In [1]:
import os
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import orth

In [2]:
# supported dataset names: 'cifar10', 'lasot', 'stl10'
dataset_name = 'cifar10'

In [3]:
# load dataset
if dataset_name == 'lasot':
    path_dataset = '/home/hyunjoon/dataset/lasot/crop/'
    list_fn = open(os.path.join(path_dataset, 'train_lasot_10class.txt')).read().splitlines()
    list_fn = [l.split(' ') for l in list_fn]
    dict_fn = {l[0]: int(l[1]) for l in list_fn}
    list_fn = [k for k in dict_fn]
    list_label = [v for v in dict_fn.values()]
    
if dataset_name == 'stl10':
    path_dataset = '/home/hyunjoon/dataset/stl-10-biased'
    list_fn = np.sort([l for l in os.listdir(path_dataset) if l.endswith('.jpg')])
    list_label = [int(os.path.splitext(fn)[0].split('_')[-1]) for fn in list_fn]
    
if dataset_name == 'cifar10':
    path_dataset = '/home/hyunjoon/dataset/cifar-10-biased'
    list_fn = np.sort([l for l in os.listdir(path_dataset) if l.endswith('.jpg')])
    list_label = [int(os.path.splitext(fn)[0].split('_')[-1]) for fn in list_fn]

In [4]:
# total number of images in the dataset
len(list_fn)

54500

In [5]:
all_imgs = [cv2.resize(cv2.imread(os.path.join(path_dataset, fn)), (32, 32)) for fn in list_fn]

In [6]:
# def color_feature(img, out_wh=(8, 8)):
#     if img.dtype == 'uint8':
#         img = img.astype(float) / 255.0
#     else:
#         img = img.astype(float)
#     r_img = cv2.resize(img, out_wh, interpolation=cv2.INTER_AREA)
#     r2 = cv2.resize(img**2, out_wh, interpolation=cv2.INTER_AREA)
#     r2 -= r_img**2
#     r2 = np.sqrt(np.maximum(r2, 0))
#     r_img = np.concatenate([r_img, 1.0 - r_img], axis=-1).ravel()
#     r2 = np.concatenate([r2, 1.0 - r2], axis=-1).ravel()
# #     r_img = np.hstack([r_img, r2])
#     r_img = np.hstack([r_img - 0.5, r2 - 0.2])
#     r_img = r_img / np.sqrt(np.sum(r_img**2))
#     return r_img

### Image feature
We use a simple color based image feature - mean and variance of pixel colors within a block.
A simple trick is that we use gradient of features to make them zero-meaned ones.
We found that this works better with random projection in practice.

In [7]:
def cgrad_feature(img, out_wh=(8, 8)):
    if img.dtype == 'uint8':
        img = img.astype(float) / 255.0
    else:
        img = img.astype(float)
        
    r_img = cv2.resize(img, out_wh, interpolation=cv2.INTER_AREA)
    r2 = cv2.resize(img**2, out_wh, interpolation=cv2.INTER_AREA)
#     r_img = np.mean(r_img, axis=-1, keepdims=True)
#     r2 = np.mean(r2, axis=-1, keepdims=True)
    r2 -= r_img**2
    r2 = np.sqrt(np.maximum(r2, 0))
    
    # mean and variance feature, zero padding for gradient computation
    rr = np.pad(np.concatenate([r_img, r2]), ((1, 1), (1, 1), (0, 0)))

    # compute gradients along x- and y-axes
    rx = rr[1:-1, :-2, :] - rr[1:-1, 2:, :]
    ry = rr[:-2, 1:-1, :] - rr[2:, 1:-1, :]

    # concat and l2 normalize
    res = np.concatenate([rx, ry], axis=-1)
    res = res / np.sqrt(np.sum(res**2))
    return res

In [8]:
all_clr = np.array([cgrad_feature(img, out_wh=(4, 4)) for img in all_imgs])

### Hashing with random projection 
[Random projection](https://lovit.github.io/machine%20learning/vector%20indexing/2018/03/28/lsh/) is a widely used technique for hashing features in a [locally sensitive way](https://en.wikipedia.org/wiki/Locality-sensitive_hashing).
There are various ways for computing hash-able features after random projection, and we use a simple heuristic; binarize based on signs of random projection results.

With hash, we can get $O(kn)$ theoretical complexity, where $k$ is the number of random projection vectors (32 in this experiment).
The actual expected complexity would be $\alpha kn$, where $\alpha$ represent the average hash collision count.

In [22]:
class RandomProjector(object):
    '''
    '''
    def __init__(self, odim=32):
        self.odim = odim
        self.proj = None

    def project(self, feat):
        ndim = feat.shape[-1]
        feat = np.reshape(feat, (-1, ndim))
        
        # random projection matrix would become unstable, so reject such cases
        assert ndim >= self.odim, '{} is smaller than odim ({})'.format(ndim, self.odim)
        
        # compute the random projection matrix
        if self.proj is None:
            for _ in range(100):
                # try to get an orthonormal projection matrix
                self.proj = orth(np.random.uniform(-1, 1, (ndim, ndim)))[:, :self.odim]
                if self.proj.shape[1] == self.odim:
                    break
            else:
                # if failed to get an orthonormal one, just use a random one instead
                self.proj = np.random.uniform(-1, 1, (ndim, ndim))
                self.proj /= np.sqrt(np.sum(self.proj**2, axis=1, keepdims=True))
        
        # simple binarization
        # compute dot product between each feature and each projection basis,
        # then use its sign for the binarization
        feat_binary = np.dot(feat, self.proj) >= 0

        # generate hash key strings
        # assign hex string from each consecutive 16 bits and concatenate
        all_key = np.packbits(feat_binary, axis=-1)
        all_key = np.array(list(map(lambda row: ''.join(['{:02x}'.format(r) for r in row]), all_key)))
        
        if len(all_key) == 1:
            return all_key[0]
        else:
            return all_key

In [10]:
n_img = len(all_clr)
projector = RandomProjector(32)

In [11]:
# compute hash keys from all the features
all_clr = np.reshape(all_clr, (n_img, -1))
all_key = projector.project(all_clr)

In [12]:
# remove duplication using hash
clr_dict = {}
kept_hash = []

# count the number of dot products
dot_count = 0

fidx = np.random.permutation(np.arange(n_img))
for ii in fidx:
    key = all_key[ii]
    clr = all_clr[ii]
    if key not in clr_dict:
        clr_dict[key] = [clr]
        kept_hash.append(ii)
        continue
    
    # hash collision: compare dot-product based feature similarity
    max_sim = np.max(np.dot(clr_dict[key], clr)**50)
    
    # increase the number of dot product count
    dot_count += len(clr_dict[key])
    
    # keep if not a duplicated one
    if max_sim < 0.5:
        clr_dict[key].append(clr)
        kept_hash.append(ii)

In [13]:
# average number of dot product (collision) counts
dot_count / n_img

18.8711376146789

In [14]:
kept_fn = [list_fn[ii] for ii in kept_hash]
kept_label = [list_label[ii] for ii in kept_hash]

# total number of images after removal
len(kept_fn)

12850

In [15]:
# save the list after duplication removal
classes = np.zeros((10), dtype=np.int32)
with open('train_kept_{}_hash.txt'.format(dataset_name), 'w') as fh:
    for fn, cid in zip(kept_fn, kept_label):
        classes[cid] += 1
        fh.write('{} {}\n'.format(fn, cid))
        
# see how many images are kept, per-class, after the duplication removal
classes

array([8368,  498,  492,  499,  499,  499,  500,  500,  495,  500],
      dtype=int32)

### Baseline duplication removal
This is the baseline method, with $O(n^2)$ complexity.
We use NMS to remove duplication in a sequential way.

In [16]:
removed_idx = []
fidx = np.random.permutation(np.arange(len(all_clr)))
clrs = np.reshape(np.array(all_clr), (len(all_imgs), -1))
kept_naive = []
remaining = fidx.copy()

# count the number of dot products
dot_count = 0

collapse = {}
for ii in range(len(all_clr)):
    if len(remaining) == 0:
        break
    pidx = remaining[0]
    kept_naive.append(pidx)
    if len(remaining) == 1:
        break
    
    p_clr = clrs[pidx]
    
    # dot product between a pivot feature and all the remaining ones
    sims_clr = np.dot(clrs[remaining], p_clr)**50
    ridx = np.where(sims_clr < 0.5)[0]
    
    # increase the number of dot product counts
    dot_count += len(remaining)
    
    remaining = remaining[ridx]

In [17]:
# average number of dot product counts
dot_count / n_img

1475.4708440366971

In [18]:
kept_fn = [list_fn[ii] for ii in kept_naive]
kept_label = [list_label[ii] for ii in kept_naive]

# total number of images after removal
len(kept_fn)

7488

In [19]:
classes = np.zeros((10), dtype=np.int32)
with open('train_kept_{}_naive.txt'.format(dataset_name), 'w') as fh:
    for fn, cid in zip(kept_fn, kept_label):
        classes[cid] += 1
        fh.write('{} {}\n'.format(fn, cid))
        
# see how many images are kept, per-class, after the duplication removal
classes

array([3031,  494,  483,  499,  496,  497,  499,  499,  490,  500],
      dtype=int32)

In [20]:
total_classes = np.zeros((10), dtype=np.int32)
for cid in list_label:
    total_classes[cid] += 1
total_classes

array([50000,   500,   500,   500,   500,   500,   500,   500,   500,
         500], dtype=int32)

At this point, you will have two annotation files, `train_kept_{}_naive.txt` and `train_kept_{}_hash.txt`.
Move the two files to your dataset directory to train classifiers and compare performances.